## FVA origins

The `FVA` term is the amount of $s$'s exports to $r$ whose value added originated outside of $s$. Alternatively, one might wish to calculate the actual origins of such value added. The following decomposition does just that: it breaks down $s$'s `FVA` exports into its value added origins, indexed by $u$.

In [ ]:
import numpy as np
import pandas as pd
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

In [ ]:
input, output = 'adb-mrio.parquet', 'ta-fva.parquet'
# input, output = 'adb-mrio62.parquet', 'ta62-fva.parquet'
# input, output = 'adb-mrio62-const.parquet', 'ta62-const-fva.parquet'

In [ ]:
sectors = pd.read_excel('../data/raw/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
DF = pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_FC = subset(v, -s) @ subset(Bnots, -s, s)
        FVA = VB_FC @ subset(E, s, -s)
        FVAorigin = np.transpose(diagvec(subset(v, -s))) @ subset(Bnots, -s, s) @ np.sum(subset(E, s, -s), axis=1)
        
        DFs = pd.DataFrame({
            't': year,
            'u': np.setdiff1d(np.arange(1, G+1), s),
            's': s,
            'FVA': np.sum(FVA),
            'FVAorigin': FVAorigin,
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

DF.to_parquet(f'../data/adhoc/{output}', index=False)